In [1]:
import numpy as np
import sympy as sp
from numba import njit,jit,prange

In [2]:
# 变步长龙贝格积分
def step_change_romberg_int(int_fun, s,d, nn, ep):
    error = 10.
    h = (d-s)/nn
    r2 = 0
    r1 = 0
    
    # 1分梯形积分
    sum_f1 = lambda x:(int_fun(x)+int_fun(x+h))*h/2
    t1 = np.sum( sum_f1(np.arange(s, d-h, h)) )
    # 2分梯形积分
    sum_f2 = lambda x:int_fun(x+h/2)
    t2 = t1/2 + h/2 *np.sum(sum_f2(np.arange(s, d-h, h)+h/2))
    # 1分辛普森积分
    s1 = t2 + 1/3*(t2-t1)
    # 4分梯形积分
    sum_f4 = lambda x:int_fun(x+h/4)
    t4 = t2/2 + h/8 *np.sum(sum_f4(np.arange(s, d-h/4, h/4)+h/4))
    # 2分辛普森积分
    s2 = t4 + 1/3*(t4-t2)
    # 1分柯斯特积分
    c1 = s2 + 1/15*(s2-s1)
    # 8分梯形积分
    sum_f8 = lambda x:int_fun(x+h/8)
    t8 = t4/2 + h/16 *np.sum(sum_f8(np.arange(s, d-h/8, h/8)+h/8))
    # 4分辛普森积分
    s4 = t8 + 1/3*(t8-t4)
    # 2分柯斯特积分
    c2 = s4 + 1/15*(s4-s2)
    # 1分隆贝格积分
    r1 = c2 + 1/63*(c2-c1)
    
    while error>=ep:
        h/=2
        t4, s2, c1, r1 = t8, s4, c2, r2
        t8 = t4/2 + h/16 *np.sum(sum_f8(np.arange(s, d-h/8, h/8)+h/8))
        # 4分辛普森积分
        s4 = t8 + 1/3*(t8-t4)
        # 2分柯斯特积分
        c2 = s4 + 1/15*(s4-s2)
        # 2分隆贝格积分
        r2 = c2 + 1/63*(c2-c1)
        # 计算误差
        if(np.abs(r2)<=1):
            # 计算绝对误差
            error = np.abs(r2 - r1)
        else:
            # 计算相对误差
            error = np.abs((r2-r1)/r2)
    return r2

# 变步长辛普森积分
def step_change_simpson_int(int_fun, s,d, nn, ep):
    error = 10.
    h = (d-s)/nn

    # 1分梯形积分
    sum_f1 = lambda x:(int_fun(x)+int_fun(x+h))*h/2
    t1 = np.sum( sum_f1(np.arange(s, d-h, h)) )
    # 2分梯形积分
    sum_f = lambda x:int_fun(x+h/2)
    t2 = t1/2 + h/2 *np.sum(sum_f(np.arange(s, d-h, h)+h/2))
    # 1分辛普森积分
    s1 = t2 + 1/3*(t2-t1)
    
    while error>=ep:
        h/=2
        t4 = t2/2 + h/2 *np.sum(sum_f(np.arange(s, d-h, h)+h/2))
        # 2分辛普森积分
        s2 = t4 + 1/3*(t4-t2)
        # 计算误差
        if(np.abs(s2)<=1):
            # 计算绝对误差
            error = np.abs(s2 - s1)
        else:
            # 计算相对误差
            error = np.abs((s2-s1)/s2)
        s1 = s2
        t2 = t4
    return s2

应用变步长辛普森法计算积分
$$
G=\int_{0}^{1} \frac{\arctan x}{x} d x
$$
此积分值称为Catalan常数，G的真值为0.915965，误差$10^{-5}$

In [13]:
print(step_change_simpson_int(lambda x:np.arctan(x)/x, np.finfo(np.float64).eps, 1., 100, 1e-5))

0.9159547825499891


应用龙贝格法计算积分
$$
I=\int_{0.1}^{0.6} \frac{0.02792(2-x)}{(1.449 x+1)^{0.8}(1-x)^{1.2} x} d x
$$
要求误差不超过$10^{-5}$

In [15]:
print(step_change_romberg_int(lambda x:(0.02792*(2-x))/((1.449*x+1)**0.8*(1-x)**1.2*x), 0.1, 0.6, 100, 1e-5))

0.10040403662017915
